Installing dependencies

In [38]:
%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

skipping


Trying out TradingView API

In [39]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [40]:
import json
import os
from datetime import datetime
import plotly.express as px

import numpy as np
import pandas as pd
import requests
from binance.client import Client

In [41]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']
ta_api = os.environ['taapi_key']


# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
# info = client.get_symbol_info('ETHUSDT')
# print(info)

In [42]:
%%script echo skipping

def get_data_frame(symbol):
    # starttime = '6 months ago UTC'
    starttime = '1 Jan, 2017'
    interval = '1h'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])

    return df

get_data_frame('ETHUSDT')

skipping


In [43]:
%%script echo skipping

def macd_logic(symbol):

    symbol_df = get_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['200_ema'] < symbol_df['close'])), 1, 0)

    return symbol_df

macd_logic('ETHUSDT')

skipping


# BACKTESTING

Backtesting completed on 6 month historical data on 5m time frame

In [44]:
%%script echo skipping
def get_six_month_data_frame(symbol):
    # starttime = '6 months ago UTC'
    starttime = '1 Jan, 2017'
    interval = '1h'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
    return df

six_month_eth_df_raw = get_six_month_data_frame('ETHUSDT')

skipping


In [45]:
# url1 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1660629600000'
# url2 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1664229600000'
# url3 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1667829600000'

def get_bars(symbol, limit = '1000', interval = '1h'):
    df = pd.DataFrame()
    root_url = 'https://api.binance.com/api/v1/klines'
    end_times = ['1639029600000', '1642629600000', '1646229600000', '1649829600000', '1653429600000', '1657029600000', '1660629600000', '1664229600000', '1667829600000']
    for time in end_times:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval + '&limit=' + limit + '&endTime=' + time
        data = json.loads(requests.get(url).text)
        for row in data:
            del row[5:]
        df = df.append(data)
    df.columns = ['date',
                'open', 'high', 'low', 'close']
    return df.reset_index()

eth_df_raw = get_bars('ETHUSDT')

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is de

In [46]:
1642629600000 - 3600000000

1639029600000

In [47]:
%%script echo skipping

def test_macd_logic(symbol):

    symbol_df = get_bars(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['close'] > symbol_df['200_ema']) & (symbol_df['macd'] < 0) & (symbol_df['signal'] > symbol_df['macd'].shift())), 1, 0)
    
    tr_list = [(symbol_df['high'] - symbol_df['low']), (symbol_df['low'] - symbol_df['close'].shift()), (symbol_df['high'] - symbol_df['close'].shift())]

    symbol_df['TR1'] = symbol_df['high'] - symbol_df['low']
    symbol_df['prev close'] = symbol_df['close'].shift()
    symbol_df['TR2'] = symbol_df['low'] - symbol_df['close'].shift()
    symbol_df['TR3'] = symbol_df['high'] - symbol_df['close'].shift()

    symbol_df = symbol_df.iloc[200:]

    triggered_df = symbol_df.loc[symbol_df['Trigger'] == 1]

    return symbol_df

test_df = test_macd_logic('ETHUSDT')

skipping


In [48]:
%%script echo skipping

atr_indicator = "atr"
atr_endpoint = f"https://api.taapi.io/{atr_indicator}"

live_price_indicator = "price"
live_price_endpoint = f"https://api.taapi.io/{atr_indicator}"

stop_loss = 0
tp = 0
entry = 0

parameters = {
    'secret': ta_api,
    'exchange': 'binance',
    'symbol': 'ETH/USDT',
    'interval': '1H'
    } 

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

atr_test = 16.74326363683623

for index, row in test_df.iterrows():
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True

            stop_loss = row['close'] - atr_test
            tp = row['close'] + atr_test
            entry = row['close']
            
    
    else:
        if row['close'] <= stop_loss:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')
            tp = (entry + (2*atr_test))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: {profit - losses}')

skipping


In [49]:
def test_logic_2(raw_df):
    raw_df['open'] = pd.to_numeric(raw_df['open'])
    raw_df['high'] = pd.to_numeric(raw_df['high'])
    raw_df['low'] = pd.to_numeric(raw_df['low'])
    raw_df['close'] = pd.to_numeric(raw_df['close'])
    raw_df['date'] = pd.to_datetime(raw_df['date'], unit = 'ms')


    short_ema = raw_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = raw_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = raw_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    raw_df['macd'] = macd
    raw_df['signal'] = signal
    raw_df['200_ema'] = ema_200

    raw_df['TR1'] = abs(raw_df['high'] - raw_df['low'])
    raw_df['TR2'] = abs(raw_df['low'] - raw_df['close'].shift())
    raw_df['TR3'] = abs(raw_df['high'] - raw_df['close'].shift())

    raw_df['ATR'] = raw_df.apply(lambda x: row_check(x.TR1, x.TR2, x.TR3), axis=1)

    raw_df = raw_df.iloc[200:]

    raw_df = raw_df.drop(['TR1', 'TR2', 'TR3'], axis=1)

    raw_df['trigger_long'] = np.where(((raw_df['macd'] > raw_df['signal']) & (raw_df['close'] > raw_df['200_ema']) & (raw_df['macd'] < 0) & (raw_df['signal'] > raw_df['macd'].shift())), 1, 0)

    raw_df['trigger_short'] = np.where(((raw_df['signal'] > raw_df['macd']) & (raw_df['close'] < raw_df['200_ema']) & (raw_df['macd'] > 0) & (raw_df['signal'] < raw_df['macd'].shift())), 1, 0)


    return raw_df

def row_check(TR1, TR2, TR3):
    if TR1 >= TR2 and TR1 >= TR3:
        ATR = TR1
    elif TR2 >= TR1 and TR2 >= TR3:
        ATR = TR2
    else:
        ATR = TR3
    
    return ATR

In [50]:
tinkered_df = test_logic_2(get_bars('ETHUSDT'))

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_16417/2575978689.py:14: FutureWarning:

The frame.append method is de

In [51]:
tinkered_df

,index,date,open,high,low,close,macd,signal,200_ema,ATR,trigger_long,trigger_short
200,200,2021-11-05 23:00:00,4480.10,4482.04,4455.70,4474.24,-12.374320,-10.620439,4412.130882,26.34,0,0
201,201,2021-11-06 00:00:00,4474.24,4507.29,4455.88,4498.86,-10.142059,-10.524763,4412.993859,51.41,1,0
202,202,2021-11-06 01:00:00,4498.86,4511.70,4477.69,4497.42,-8.392429,-10.098296,4413.833920,34.01,1,0
203,203,2021-11-06 02:00:00,4497.42,4504.76,4473.15,4482.55,-8.112207,-9.701078,4414.517662,31.61,0,0
204,204,2021-11-06 03:00:00,4482.54,4497.44,4474.21,4491.01,-7.125342,-9.185931,4415.278780,23.23,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8995,995,2022-11-07 10:00:00,1564.10,1584.56,1563.88,1578.54,-13.407928,-12.128339,1580.258033,20.68,0,0
8996,996,2022-11-07 11:00:00,1578.53,1582.02,1575.66,1580.00,-12.353065,-12.173284,1580.255466,6.36,0,0
8997,997,2022-11-07 12:00:00,1579.99,1585.02,1576.48,1584.51,-11.026057,-11.943839,1580.297799,8.54,1,0
8998,998,2022-11-07 13:00:00,1584.50,1589.73,1575.31,1579.30,-10.276338,-11.610339,1580.287871,14.42,0,0


In [52]:
fig = px.line(tinkered_df, x="date", y="close", title='Close price of ETH/USDT, 1H timeframe')
fig.show()

In [53]:
%%script echo skipping
stop_loss = 0
tp = 0
entry = 0

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

for index, row in tinkered_df.iterrows():
    atr = 0
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True
            atr = row['ATR']
            stop_loss = row['close'] - atr
            tp = row['close'] + atr
            entry = row['close']
            print('')
            print('')
            print('OPENED TRADE')
            print(f'Bought 1 ETH @ {entry}')
            print(f'SL: {stop_loss}, TP1: {entry + atr}, TP2: {entry + 2*atr}')
            print('')
            
    
    else:
        if row['close'] <= stop_loss and tp1_hit == False:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print(f'Current P/L: ${profit-losses}')
            print('')

        if row['close'] <= stop_loss and tp1_hit == True:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)/2
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print(f'Current P/L: ${profit-losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print(f'Current P/L: ${profit-losses}')
            print('')
            tp = (entry + (2*atr))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print(f'Current P/L: ${profit-losses}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: ${profit - losses}')

skipping


In [54]:
%%script echo skipping

balance = 10000
stop_loss = 0
tp1 = 0
tp2 = 0
entry = 0

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

for index,row in tinkered_df.iterrows():
    if row['Trigger'] == 1 and in_trade == False:

        atr = round(row['ATR'],2)
        entry = round(row['close'],2)
        stop_loss = round(entry - (1*atr),2)
        tp1 = round(entry + (1*atr),2)
        tp2 = round(entry + (2*atr),2)
        ema_200 = round(row['200_ema'],2)
        in_trade = True
        tp1_hit = False
        
        print(f'\n\nEntry Price: ${entry}\nStop Loss: ${stop_loss}\nATR: ${atr}\n200 EMA: ${ema_200}\nTP1: ${tp1}\nTP2: ${tp2}\n\n')

    # if row['Trigger_Down'] == 1 and in_trade == False:
    #     atr = round(row['ATR'],2)
    #     entry = round(row['close'],2)
    #     stop_loss = round(entry - (1*atr),2)
    #     tp1 = round(entry + (1*atr),2)
    #     tp2 = round(entry + (2*atr),2)
    #     ema_200 = round(row['200_ema'],2)
    #     in_trade = True
    #     tp1_hit = False

    #     print(f'\n\nEntry Price: ${entry}\nStop Loss: ${stop_loss}\nATR: ${atr}\n200 EMA: ${ema_200}\nTP1: ${tp1}\nTP2: ${tp2}\n\n')
    
    if in_trade == True:
        if row['low'] <= stop_loss and tp1_hit == False:
            loss += 1
            losses += (entry - stop_loss)
            in_trade = False
            print('SL Hit.')
            print(f'Entry: ${entry}')
            print(f'Stop Loss: ${stop_loss}')
            print(f'Losses: ${round(entry - stop_loss,2)}')
            print(f'Current P/L: ${round(profit-losses,2)}')
            print('')
            continue
        
        elif row['low'] <= stop_loss and tp1_hit == True:
            loss += 1
            losses += (entry - stop_loss)/2
            in_trade = False
            print('SL Hit.')
            print(f'Entry: ${entry}')
            print(f'Stop Loss: ${stop_loss}')
            print(f'Losses: ${round((entry - stop_loss)/2,2)}')
            print(f'Current P/L: ${round(profit-losses,2)}')
            print('')
            continue
        
        elif row['high'] >= tp2:
            if tp1_hit == True:
                profit += round((tp2 - entry)/2,2)
                in_trade = False
                tp1_hit = False
                print('TP2 HIT')
                print(f'Earnings: ${round((tp2 - entry)/2,2)}')
                print(f'Current P/L: ${round(profit-losses,2)}')
                print('')
                continue
            else:
                profit += round((((tp2 - entry)/2)+((tp1 - entry)/2)),2)
                win += 1
                in_trade = False
                tp1_hit = False
                print('TP1 & TP2 HIT')
                print(f'Earnings: ${round(((tp2 - entry)/2)+((tp1 - entry)/2),2)}')
                print(f'Current P/L: ${round(profit-losses,2)}')
                print('')
                continue
        elif tp1_hit == False and row['high'] >= tp1:
            profit += round((tp1 - entry)/2,2)
            win += 1
            tp1_hit = True
            print('TP1 HIT')
            print(f'Earnings: ${round((tp1 - entry)/2,2)}')
            print(f'Current P/L: ${round(profit-losses,2)}')
            print('')
            continue
        else:
            continue

print(f'\n\n\n\n\n\nWinning Trades: {win}\nLosing Trades: {loss}\nProfit: ${round(profit,2)}\nLosses: ${round(losses,2)}\nP/l: ${round(profit - losses,2)}\nWin Rate: {round((win/(win+loss))*100,2)}%')


skipping


In [56]:
balance = 10000
in_long = False
in_short = False
tp1_hit = False

entry_price = 0
entry_amount = 0
stop_loss = 0
tp1 = 0
tp2 = 0

wins = 0
loss = 0
profit = 0
losses = 0

for index,row in tinkered_df.iterrows():
    if row['trigger_long'] == 1 and in_long == False and in_short == False:
        entry_amount = 0.05 * balance
        balance -= entry_amount
        atr = round(row['ATR'],2)
        entry_price = round(row['close'],2)
        stop_loss = round(entry_price - (1*atr),2)
        tp1 = round(entry_price + (1*atr),2)
        tp2 = round(entry_price + (2*atr),2)
        ema_200 = round(row['200_ema'],2)
        in_long = True
        tp1_hit = False

        print(f'\n\nLONG TRADE\nEntry Amount: ${entry_amount}\nEntry Price: ${entry_price}\nStop Loss: ${stop_loss}\nATR: ${atr}\n200 EMA: ${ema_200}\nTP1: ${tp1}\nTP2: ${tp2}\n\n')

    elif row['trigger_short'] == 1 and in_long == False and in_short == False:
        entry_amount = 0.05 * balance
        balance -= entry_amount
        atr = round(row['ATR'],2)
        entry_price = round(row['close'],2)
        stop_loss = round(entry_price + (1*atr),2)
        tp1 = round(entry_price - (1*atr),2)
        tp2 = round(entry_price - (2*atr),2)
        ema_200 = round(row['200_ema'],2)
        in_short = True
        tp1_hit = False

        print(f'\n\nSHORT TRADE\nEntry Price: ${entry_price}\nStop Loss: ${stop_loss}\nATR: ${atr}\n200 EMA: ${ema_200}\nTP1: ${tp1}\nTP2: ${tp2}\n\n')

    
    if in_long == True:
        if row['low'] <= stop_loss and tp1_hit == False:
            loss += 1
            losses += (entry_price - stop_loss)
            in_trade = False
            print('SL Hit.')
            print(f'Entry: ${entry_price}')
            print(f'Stop Loss: ${stop_loss}')
            print(f'Losses: ${round(entry_price - stop_loss,2)}')
            print(f'Current P/L: ${round(profit-losses,2)}')
            print('')
            continue



LONG TRADE
Entry Amount: $500.0
Entry Price: $4498.86
Stop Loss: $4447.45
ATR: $51.41
200 EMA: $4412.99
TP1: $4550.27
TP2: $4601.68


SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-51.41

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-102.82

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-154.23

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-205.64

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-257.05

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-308.46

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-359.87

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-411.28

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-462.69

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Losses: $51.41
Current P/L: $-514.1

SL Hit.
Entry: $4498.86
Stop Loss: $4447.45
Lo